## 평가 환경 설정

1. 패키지 설치
2. vscode 내 jupyter notebook 환경에서 input 기능 활성화

In [2]:
!pip install -r requirements.txt --quiet

In [32]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


## 필수 구현 과제
1. 사용 환경 준비

In [4]:
import os
import openai

# 환경 변수에서 호출
openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

2. 모델 로드하기

In [5]:
# 임베딩 모델
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002", api_key=openai.api_key
)

# LLM 모델
from langchain_openai import ChatOpenAI

llm_model = ChatOpenAI(model="gpt-4o-mini", api_key=openai.api_key)

3. 문서 로드하기
- 사용 문서
  - 초거대 언어모델 연구 동향.pdf
  - 박찬준 외, 「초거대 언어모델 연구 동향」, 『정보학회지』, 제41권 제11호(통권 제414호), 한국정보과학회, 2023, 8-24 
  - 출처 [¶](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11610124)

In [6]:
from langchain_community.document_loaders import PyPDFLoader

# 가져올 pdf 파일 경로
path = "documents/초거대 언어모델 연구 동향.pdf"

# 사용할 pdf loader 선택
loader = PyPDFLoader(path)

# pdf 파일 불러오기
docs = loader.load()

# 불러올 범위 설정
page = 0
start_point = 0
end_point = 500

# 결과 확인
print(f"페이지 수: {len(docs)}")
print(f"\n[페이지내용]\n{docs[page].page_content[start_point:end_point]}")
print(f"\n[metadata]\n{docs[page].metadata}\n")

페이지 수: 17

[페이지내용]
8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어
처리 (Natural Language Proce

[metadata]
{'source': 'documents/초거대 언어모델 연구 동향.pdf', 'page': 0}



4. 문서 청크로 나누기

In [7]:
# 파라미터 설정
CHUNK_INDEX = 0
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
SEPERATOR = "\n"

i .`CharacterTextSplitter`

In [8]:
from langchain.text_splitter import CharacterTextSplitter

# 문서 분할기 설정
splitter = CharacterTextSplitter(
    separator=SEPERATOR,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=True,
)

# 문서 분할
c_splits = splitter.split_documents(docs)

# 결과 확인
print(
    f"c_splits\n길이 : {len(c_splits)}\n 결과 확인 : {c_splits[CHUNK_INDEX].page_content}"
)

c_splits
길이 : 142
 결과 확인 : 8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어


ii .`RecursiveCharacterTextSplitter`

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서 분할기 설정
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)

# 문서 분할
r_splits = recursive_splitter.split_documents(docs)

# 결과 확인
print(
    f"r_splits\n길이 : {len(r_splits)}\n 결과 확인 : {r_splits[CHUNK_INDEX].page_content}"
)

r_splits
길이 : 141
 결과 확인 : 8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어


- **CharacterTextSplitter**
  - 기본적인 분할 방식
  - 구분자를 기준으로 청크 단위로 분할
  
- **RecursiveCharacterTextSplitter**
  - 단락-문장-단어 순서로 재귀적으로 분할
  - 여러번의 분할로 작은 덩어리 생성
  - 텍스트가 너무 크거나 복잡할 때 유용

In [10]:
# Sample 10개 테스트

# def sample_printer(splits_1, splits_2, n):
#     for i in range(n):
#         print(f"Sample 생성중...")
#         print(f"Sample_1 {i} \n {splits_1[i].page_content}")
#         print(f"Sample_2 {i} \n {splits_2[i].page_content}")

# sample_printer(c_splits, r_splits, 10)

5. 벡터 임베딩 생성

In [11]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002", api_key=openai.api_key)

6. 벡터 스토어 생성
- `RecursiveCharacterTextSplitter` 사용

In [12]:
# RecursiveCharacterTextSplitter 결과 사용
splits = r_splits

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index = faiss.IndexFlatL2(len(embedding_model.embed_query("This is Sample Text.")))
vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(splits))]
vector_store.add_documents(documents=splits, ids=uuids)

# 테스트
results_with_scores = vector_store.similarity_search_with_score(
    "RAG에 대해 이야기해주세요.", k=2, filter={"source": 'documents/초거대 언어모델 연구 동향.pdf'}
)
for res, score in results_with_scores:
    print(f"* [SIM={score:.3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.345] 16 특집원고  초거대 언어모델 연구 동향
Retrieval Augmented Generation (RAG) [95, 96, 97, 
98]이라 한다.
Other Tools L a M D A  [ 4 4 ]는 대화 애플리케이션에 
특화된 LLM으로, 검색 모듈, 계산기 및 번역기 등의 
외부 도구 호출 기능을 가지고 있다. WebGPT [99] 는 
웹 브라우저와의 상호작용을 통해 검색 쿼리에 사실 
기반의 답변과 함께 출처 정보를 제공 한다. PAL 
[100]은 Python 인터프리터를 통한 복잡한 기호 추론 
기능을 제공하며, 여러 관련 벤치마크에서 뛰어난 성
능을 보여주었다. 다양한 종류의 API (e.g., 계산기, 달
력, 검색, QA, 번역 등 단순한 API에서부터 Torch/ 
TensorFlow/HuggingFace Hub에 이르는 복잡한 API까
지) 호출 기능을 갖춘 연구들 [101, 102, 103, 104, [{'source': 'documents/초거대 언어모델 연구 동향.pdf', 'page': 8}]
* [SIM=0.424] 같은 문자라면 같은 밀집 벡터로 표현하기 때문에 문
맥 정보를 반영하지 못한다는 한계를 지닌다.
문맥기반 언어모델 연구 문맥 정보를 반영하여 언
어를 표현하기 위해, 텍스트 내의 정보를 이용하는 
RNN (Recurrent Neural Network) 이 등장했다. 그러나, 
RNN은 입력 텍스트의 길이가 길어질수록 앞쪽에 위 [{'source': 'documents/초거대 언어모델 연구 동향.pdf', 'page': 1}]


7. FAISS를 Retriever로 변환

In [13]:
from langchain_community.vectorstores import FAISS

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [14]:
# 리트리버 테스트
# query = "RAG에 대해 이야기해주세요."

# retriever = vector_store.as_retriever(search_type="similarity")
# results = retriever.invoke(query)
# for result in results:
#     print(f"Source: {result.metadata['source']} | Page: {result.metadata['page']}")
#     print(f"Content: {result.page_content.replace('\n', ' ')}\n")

8. 프롬프트 템플릿 정의

In [15]:
from langchain_core.prompts import ChatPromptTemplate

contextual_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the question using only the following context."),
        ("user", "Context: {context}\\n\\nQuestion: {question}"),
    ]
)

9. RAG 체인 구성

In [16]:
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs


class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = [doc.page_content.replace("\n", " ") for doc in inputs]
            
        else:
            context_text = inputs

        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text, question=inputs.get("question", "")
        )
        return formatted_prompt


# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.invoke(query)
        return response_docs

In [17]:
PROMPT_BASELINE = "Answer the question using only the following context."
REFERENCE_BASELINE = "check user qestion"

def RAGChainMake(VECTOR_STORE, KEY, LLM_MODEL='gpt-3.5-turbo', PROMPT=PROMPT_BASELINE, REFERENCE=REFERENCE_BASELINE, **kwargs):
    """
    RAG 기법을 이용한 대화형 LLM 답변 체인 생성 (히스토리 기억 및 동적 대화 기능 포함)

    VECTOR_STORE : Retriever가 검색할 벡터 스토어
    KEY : API Key
    LLM_MODEL : LLM 모델명
    PROMPT    : 시스템 초기 프롬프트 (기본값 설정)
    REFERENCE : 추가 문맥 정보 (선택 사항)
    """
    # 벡터 스토어에서 유사한 문맥 검색
    retriever = VECTOR_STORE.as_retriever(
        search_type="similarity", search_kwargs={"k": 1}
    )

    # API 키 설정
    openai.api_key = os.environ.get(KEY)
    llm_model = ChatOpenAI(
        model=LLM_MODEL,
        api_key=openai.api_key,
    )

    # 대화형 프롬프트 생성
    contextual_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", f'{PROMPT} \n\n reference : {REFERENCE}'),
            ("user", "Context: {context}\n\nQuestion: {question}"),
        ]
    )

    # RAG 체인 설정
    rag_chain_debug = {
        "context": RetrieverWrapper(retriever),
        "prompt": ContextToPrompt(contextual_prompt),
        "llm": llm_model,
    }

    return rag_chain_debug

10. 챗봇 구동 확인

In [18]:
def RAG_Coversation(CHAIN, **kwargs):
    """
    사용자로부터 질문을 받아 RAG 체인 기반으로 답변을 생성하는 대화형 함수
    전체 대화 결과를 리스트에 저장
    """
    print("대화를 시작합니다. 종료하려면 'exit'를 입력하세요.\n")

    conversation_history = []  # 대화 기록을 저장할 리스트

    while True:
        print("========================")
        query = input("질문을 입력하세요 : ")

        if query.lower() == "exit":
            print("대화를 종료합니다.")
            break

        # 1. Retriever로 관련 문서 검색
        response_docs = CHAIN["context"].invoke({"question": query})

        # 2. 문서를 프롬프트로 변환
        prompt_messages = CHAIN["prompt"].invoke(
            {"context": response_docs, "question": query}
        )

        # 3. LLM으로 응답 생성
        response = CHAIN["llm"].invoke(prompt_messages)

        print("\n답변:")
        print(response.content)

        conversation_history.append({"question": query, "answer": response.content})

>질문 = '업스테이지 solar 모델에 대해서 알려주세요.'

In [27]:
key = "MY_OPENAI_API_KEY"
llm_model = 'gpt-3.5-turbo'
chatbot_mk4 = RAGChainMake(vector_store, key, llm_model)
RAG_Coversation(chatbot_mk4)

대화를 시작합니다. 종료하려면 'exit'를 입력하세요.


답변:
업스테이지의 Solar 모델은 Llama2를 파인튜닝하여 개발된 Solar-0-70b 모델입니다. 이 모델은 한국어와 영어를 지원하며 글로벌 LLM 플랫폼 중 하나인 Poe.com에서 서비스되고 있습니다.
대화를 종료합니다.


>같은 질문을 일반 chat gpt 혹은 Gemini에 질문해보고 답변을 비교해보고, 왜 RAG이 필요한지 간단히 markdown으로 서술해주세요.

In [30]:
from openai import OpenAI
QUESTION = '업스테이지 solar 모델에 대해서 알려주세요.'
model_3 = 'gpt-3.5-turbo'
client = OpenAI(api_key=openai.api_key)
completion = client.chat.completions.create(
        model=model_3,
        messages=[
            {"role": "user", "content": QUESTION},
        ],
        temperature=0.0,
    )
gpt_3 = completion.choices[0].message.content
print(gpt_3)

업스테이지 솔라 모델은 한국의 걸그룹 마마무의 솔라가 모델로 활동하는 뷰티 브랜드입니다. 업스테이지는 솔라의 개성과 매력을 반영한 제품들을 선보이고 있으며, 솔라의 이미지와 함께 다양한 메이크업 제품을 출시하고 있습니다. 솔라의 화려하고 독특한 스타일을 따라가는 소비자들에게 인기가 있는 브랜드로, 솔라의 팬들뿐만 아니라 메이크업을 즐기는 많은 사람들에게 사랑받고 있습니다.


In [31]:
model_4 = 'gpt-4o-mini'
client = OpenAI(api_key=openai.api_key)
completion = client.chat.completions.create(
        model=model_4,
        messages=[
            {"role": "user", "content": QUESTION},
        ],
        temperature=0.0,
    )
gpt_4 = completion.choices[0].message.content
print(gpt_4)

업스테이지(Upstage)에서 개발한 Solar 모델은 자연어 처리(NLP)와 관련된 다양한 작업을 수행할 수 있는 인공지능 모델입니다. 이 모델은 대규모 데이터셋을 기반으로 학습되어, 텍스트 생성, 요약, 번역, 질문 응답 등 여러 가지 언어 관련 작업에서 높은 성능을 발휘합니다.

Solar 모델은 특히 한국어 처리에 최적화되어 있으며, 한국어의 문법적 특성과 어휘를 잘 이해하고 있습니다. 이를 통해 한국어로 된 다양한 텍스트에 대한 이해와 생성이 가능하며, 사용자에게 보다 자연스러운 대화 경험을 제공합니다.

업스테이지는 Solar 모델을 통해 기업이나 개발자들이 AI 기반의 솔루션을 쉽게 구축할 수 있도록 지원하고 있으며, 다양한 API와 도구를 제공하여 사용자가 자신의 필요에 맞게 모델을 활용할 수 있도록 하고 있습니다.

더 구체적인 기능이나 사용 사례에 대해 알고 싶으시면 말씀해 주세요!


## 답변 비교
- 질문 : 업스테이지의 solar 모델에 대해 설명해줘.
- 확인 사항 : solar 모델은 2023년에 개발된 업스테이지의 최신 모델

### GPT-3.5-turbo
- 최신 정보를 반영하지 못하고 할루시네이션 발생

### GPT-4o-mini
- 최신 정보를 반영하여 solar 모델에 대한 정보를 제공
- 상세한 정보를 반영하고 있지 않고 대략적인 정보만 제공

### RAG + GPT-3.5-turbo
- GPT-3.5-turbo에 없는 정보를 RAG 검색을 이용하여 답변
- 업스테이지에서 작성한 문서르 참고했기 떄문에 모델명 등의 상세한 정보를 제공

## 결론
> RAG를 이용하면 찾는 정보가 LLM의 데이터에 없는 경우나 있더라도 상세한 내용을 설명하지 못하는 경우에 보다 자세하고 신뢰할 수 있는 답변을 생성 할 수 있다.

## 도전 구현 과제
1. LangSmith의 Prompt Library를 참고하여 prompt engineering을 수행

RAG_Module 패키지 사용법
- Clone한 경우 바로 사용가능

1. RAG_Module 폴더 다운로드
2. RAG_Module 디렉토리로 이동
3. `pip install .`

In [33]:
from RAG_Module.RAG_Params import *
from RAG_Module.PDF_Loader import PDFLoader
from RAG_Module.VecotorStore_Utils import VectorStoreReturn
from RAG_Module.RAG_Chain import *
from RAG_Module.Prompt_Engineering import *

rag_setting = RAGParams(
    KEY="MY_OPENAI_API_KEY",
    EBD_MODEL="text-embedding-ada-002",
    LLM_MODEL="gpt-3.5-turbo",
    PDF_PATH="documents/초거대 언어모델 연구 동향.pdf",
    SAVE_PATH=None,
    IS_SAFE=True,
    CHUNK_SIZE=500,
    CHUNK_OVERLAP=100,
)

prompt_setting = PromptParams(
    KEY="MY_OPENAI_API_KEY",
    LLM_MODEL="gpt-4o-mini",
    PROMPT_PATH="Prompts/",
    PROMPT_NAME=None,
    PROMPT_EXTENSION="txt",
    RESULT_PATH="Results/",
    RESULT_EXTENSION="txt",
)

# 문서 불러오기 및 분할
docs = PDFLoader(rag_setting)

# 벡터 스토어 생성
vector_store = VectorStoreReturn(docs, rag_setting)

# 프롬프트 1 작성 및 저장
prompt_1 = "Answer the question using only the following context."
PromptSave(prompt_1, prompt_setting, PROMPT_NAME='prompt_1')

# 프롬프트 2 작성 및 저장
prompt_2 = """

you are specialist of large language model
answer question
refer to context qiven in question

"""
PromptSave(prompt_2, prompt_setting, PROMPT_NAME='prompt_2')

# 프롬프트 3 작성 및 저장

## shot 기법 사용을 위한 shot 제작용 프롬프트 생성
shot_template = TemplateParams(
    PERSONA="specialist of large language model",
    LANG="only in korean",
    TONE="professional",
    PERPOSE="study large language model",
    HOW_WRITE="itemization",
    CONDITION="""

    <must obey>
    answer is about large language model
    answer that you do not know what you do not know
    </must obey>

    if you canspecify the date standard of the information
    if you can identify the original source of the document you referenced, write in APA format
    """,
    REFERENCE="only the latest information")
shot_prompt = PromptTemplate(shot_template)

## shot 생성 - 상위 모델인 gpt-4o-mini르 사용, 답변 방식을 참고하도록 유도
question = "gpt-4에 대해서 설명해줘"
shot = LLMSupport(question, prompt_setting, shot_prompt)

## 프롬프트 3 작성 및 저장
template_setting = TemplateParams(
    PERSONA="specialist of large language model",
    LANG="only in korean",
    TONE="professional",
    PERPOSE="study large language model",
    HOW_WRITE="itemization",
    CONDITION="""

    <must obey>
    answer is about large language model
    answer that you do not know what you do not know
    </must obey>

    prioritize the context in question
    specify if there are any new information

    if you can identify the original source of the document you referenced, write in APA format
    """,
    REFERENCE=f"""

    <answer format sample>
    {shot}
    </answer format>
    
    refer to context given in the question",
    """
)
prompt_3 = PromptTemplate(template_setting)
PromptSave(prompt_3, prompt_setting, PROMPT_NAME='prompt_3')

# 저장된 프롬프트 1, 2, 3 불러오기 및 결과 저장
## QUESTION 리스트 형태로 입력
QUESTION = ["업스테이지의 solar 모델에 대해 설명해줘.",]
AutoChain(prompt_setting, vector_store, QUESTION)


Load PDF.....
Load Complete....!
Split docs.....
Split Complete....!

Authenticate API KEY....
Authenticate Complete....!
Set up Embedding model....
Set up Complete....!
Initiate FAISS instance....
Return Vector Store....!

prompt_1 이/가 Prompts/ 에 저장되었습니다.

prompt_2 이/가 Prompts/ 에 저장되었습니다.

load Prompt Template....

    필요 없는 부분은 공백('')으로 작성
    영어 사용, 명령어 사용
    리스트, 마크다운 작성 방식으로 성능을 향상 가능

    PERSONA : LLM이 수행할 역할 지정
    LANG : 답변 생성 언어
    TONE : 답변의 어조 설정
    PURPOSE : 목적 명시
    HOW_WRITE : 답변 방식 예) 개조식
    CONDITION : 추가할 조건
    REFERENCE : 참조
    
Making Prompt... 


    persona : specialist of large language model
    language : only in korean
    tone : professional
    purpose : study large language model
    how to write : itemization
    condition : 

    <must obey>
    answer is about large language model
    answer that you do not know what you do not know
    </must obey>

    if you canspecify the date standard of the information
    if you can identify the original s

```
LLM_RAG/
│
├── Prompts/
│   ├── prompt_1.txt
│   ├── prompt_2.txt
│   └── prompt_3.txt
└── Results/
    ├── prompt_1_result_24_11191548.json
    ├── prompt_2_result_24_11191548.json
    └── prompt_3_result_24_11191548.json
```

## 결과
> RAG로 정보를 제공하고 상대적으로 고성능 모델의 답변 방식을 예시로 제공하면<br>
>저성능 모델에서도 의미있는 답변을 얻을 수 있음을 확인